Análisis de algoritmos

Integrantes:
Diego Silva Madariaga

Christian Pérez Flores 

Sección:
412

Profesor:
Mladen Williams Nadinic Cruz


In [ ]:
import numpy as np
from time import process_time
import matplotlib.pyplot as plt
from cmath import exp, pi

# CCS iterativa matriz
import numpy as np
from scipy.linalg import toeplitz

def ccs_iterativo_toeplitz(x, h):
    N1 = len(x)
    N2 = len(h)
    N = N1 + N2 - 1

    # cero
    h_pad = np.concatenate([h, np.zeros(N - N2)])
    x_pad = np.concatenate([x, np.zeros(N - N1)])

    # Crear la primera columna de la matriz Toeplitz circular
    col = np.concatenate([h_pad, h_pad])[:N]
    row = np.concatenate([[h_pad[0]], h_pad[-1:0:-1]])
    toeplitz_matrix = toeplitz(col, row)  # Matriz Toeplitz circular

    # Transpuesta de la matriz Toeplitz
    HT = toeplitz_matrix.T

    # CCS: y[n] = HT @ x
    y = HT @ x_pad
    return y


# FFT recursiva
def fft_recursiva(x):
    N = len(x)
    if N == 1:
        return x
    x_even = fft_recursiva(x[::2])
    x_odd = fft_recursiva(x[1::2])
    factor = [exp(-2j * pi * k / N) * x_odd[k] for k in range(N // 2)]
    return [x_even[k] + factor[k] for k in range(N // 2)] + \
           [x_even[k] - factor[k] for k in range(N // 2)]

# IFFT recursiva
def ifft_recursiva(X):
    N = len(X)
    X_conj = np.conjugate(X)
    x = fft_recursiva(X_conj)
    x = np.conjugate(x)
    return [val / N for val in x]

# CCS usando FFT
def ccs_fft(x, h):
    N1 = len(x)
    N2 = len(h)
    N = N1 + N2 - 1
    # Aseguramos que el tamaño sea potencia de 2
    N_pot2 = 2**int(np.ceil(np.log2(N)))
    x_pad = np.concatenate([x, np.zeros(N_pot2 - N1)])
    h_pad = np.concatenate([h, np.zeros(N_pot2 - N2)])

    X = fft_recursiva(x_pad)
    H = fft_recursiva(h_pad)
    Y = [X[k] * H[k] for k in range(N_pot2)]
    y = ifft_recursiva(Y)
    return np.array(y[:N])  # recortamos al tamaño real de la convolución

# Comparación de tiempos
valores_N = [2**i for i in range(2, 14)]
tiempos_iterativo = []
tiempos_fft = []

for N in valores_N:
    x = np.random.uniform(-1, 1, N)
    h = np.random.uniform(-1, 1, N)

    # CCS Iterativo
    start = process_time()
    _ = ccs_iterativo_toeplitz(x, h)

    end = process_time()
    tiempos_iterativo.append(end - start)

    # CCS con FFT
    start = process_time()
    _ = ccs_fft(x, h)
    end = process_time()
    tiempos_fft.append(end - start)

# Gráfica
plt.figure(figsize=(10, 6))
plt.loglog(valores_N, tiempos_iterativo, 'o-', label='CCS Iterativo (O(N²))')
plt.loglog(valores_N, tiempos_fft, 's-', label='CCS FFT (O(N log N))')
plt.xlabel('Tamaño N (log scale)')
plt.ylabel('Tiempo de ejecución (s) (log scale)')
plt.title('Comparación de tiempo: CCS Iterativo vs CCS FFT')
plt.legend()
plt.grid(True, which='both', linestyle='--')
plt.show()

: 